## MEGAに出力画像とワークフローを保存する

In [ ]:
!pip install python-dotenv

import os
import subprocess
from dotenv import load_dotenv
from pathlib import Path

# === 環境準備 ===
load_dotenv(os.path.expanduser("~/.env"))
MEGA_EMAIL = os.getenv("MEGA_EMAIL")
MEGA_PASSWORD = os.getenv("MEGA_PASSWORD")

if not MEGA_EMAIL or not MEGA_PASSWORD:
    raise RuntimeError("環境変数 MEGA_EMAIL または MEGA_PASSWORD が設定されていません")

# === 共通関数群 ===

def run_command(cmd_list):
    result = subprocess.run(cmd_list, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"コマンド失敗: {' '.join(cmd_list)}\n{result.stderr}")
    return result.stdout

def mega_login(email, password):
    print("🔐 MEGAログイン中...")
    print(run_command(['mega-login', email, password]))

def mega_logout():
    print("🔓 MEGAログアウト中...")
    run_command(['mega-logout'])

def mega_list(remote_dir):
    try:
        output = run_command(['mega-ls', remote_dir])
        files = [line.strip() for line in output.strip().splitlines() if line.strip()]
        return files
    except RuntimeError as e:
        print(f"[警告] リモートディレクトリ {remote_dir} が存在しないか取得失敗: {e}")
        return []

def mega_upload(local_path, remote_dir):
    local_path = str(local_path)
    try:
        print(f"✅ アップロード: {local_path} → {remote_dir}")
        print(run_command(['mega-put', local_path, remote_dir]))
    except RuntimeError as e:
        print(f"❌ アップロード失敗: {e}")

# === ログイン実行 ===
mega_login(MEGA_EMAIL, MEGA_PASSWORD)

# === PNGファイルのアップロード（重複除外） ===
remote_png_dir = "/ComfyUI/output/"
output_dir = Path("ComfyUI/output")
png_files = sorted(output_dir.glob("*.png"), key=lambda p: p.stat().st_mtime, reverse=True)

remote_png_names = mega_list(remote_png_dir)

if png_files:
    for png in png_files:
        if png.name in remote_png_names:
            print(f"🦘 スキップ（既に存在）: {png.name}")
        else:
            mega_upload(png, remote_png_dir)
else:
    print("📁 PNGファイルが見つかりません")

# === JSONファイルのアップロード（重複除外） ===
remote_json_dir = "/ComfyUI/user/default/workflows/"
workflow_dir = Path("ComfyUI/user/default/workflows")
workflow_files = list(workflow_dir.glob("*.json"))

remote_json_names = mega_list(remote_json_dir)

if workflow_files:
    for wf in workflow_files:
        mega_upload(wf, remote_json_dir)
else:
    print("📁 JSONファイルが見つかりません")

# === ログアウト ===
mega_logout()
